## Results and conclusions:

Model is rather good at forecasting future levels of traffic, but in this form it fails to capture seasonality.


- Even very simple models were able to achieve quite good results in just a few epochs.
- However they tend to predict the same value for all 7 predicted days
- This means that initial models failed to include seasonalities and more complex patterns
- This showed that modelling general level of future traffic is quite easy,
- thus while choosing model architecture and tuning the hyperparameters we were focusing mostly on tackling more challenging task which was learning seasonalities and complex patterns.

In [ ]:
import pickle
import numpy as np
from project_functions.sample_feed_v1_multi import SampleFeed
import matplotlib.pyplot as plt

In [ ]:
# Global parameters

TRAINING_WINDOW_SIZE = 90
PREDICTED_WINDOW_SIZE = 7
N_SAMPLES = 4
N_EPOCHS = 30
BATCH_SIZE = 8

In [ ]:
# Build test dataset

features_test = dict(np.load("data/features_test.npz", allow_pickle=True))

with open("data/visits_scaler_test.pk", "rb") as file:
    visits_scaler_test = pickle.load(file)

sample_feed = SampleFeed(
    training_window_size = TRAINING_WINDOW_SIZE,
    predicted_window_size = PREDICTED_WINDOW_SIZE,
    samples_per_epoch = N_SAMPLES
    )

Xy_test = sample_feed.random_sample_array(features_test, samples_per_page=1, shuffle=False, seed=0)

In [ ]:
from keras.models import  load_model

# Select and load model

model = load_model("models/checkpoints/122830-0.0808.keras")

In [ ]:
model.evaluate(*Xy_test)

In [ ]:
y_pred = model.predict(Xy_test[0])

X_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[0][0][:,:,0])

y_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[1])

y_pred_rescaled = visits_scaler_test. \
    inverse_transform(y_pred)

In [ ]:
x_subplots = 5
y_subplots = 5

fig, axs = plt.subplots(x_subplots, y_subplots, sharex=True, sharey=False, figsize=(4*x_subplots,4*y_subplots))

ips = iter(range(Xy_test[0][0].shape[0]))

for ix in range(x_subplots):
    for iy in range(y_subplots):
        
        ip = next(ips)
        
        X_emp_i = X_test_rescaled[ip,:]
        y_emp_i = y_test_rescaled[ip,:]
        y_pred_i = y_pred_rescaled[ip,:]

        xy_emp = np.concatenate([X_emp_i, y_emp_i.reshape(PREDICTED_WINDOW_SIZE)])
        xy_pred = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_i.reshape(PREDICTED_WINDOW_SIZE)])

        axs[ix,iy].plot(xy_emp, color="black")
        axs[ix,iy].plot(xy_pred, color="blue")

In [ ]:
def one_page_results(no):

    X_emp_i = X_test_rescaled[no,:]
    y_emp_i = y_test_rescaled[no,:]
    y_pred_i = y_pred_rescaled[no,:]

    xy_emp = np.concatenate([X_emp_i, y_emp_i.reshape(PREDICTED_WINDOW_SIZE)])
    xy_pred = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_i.reshape(PREDICTED_WINDOW_SIZE)])

    return xy_emp, xy_pred

def results_gen():
    for ip in range(Xy_test[0][0].shape[0]):
        yield one_page_results(ip)

def plot_results(xy_scaled_emp, xy_scaled_pred):

    print(xy_scaled_pred[-PREDICTED_WINDOW_SIZE:])

    plt.figure(figsize=(20, 8))
    plt.plot(xy_scaled_emp, color="black")
    plt.plot(xy_scaled_pred, color="blue")
    plt.show()

results_show_iter = results_gen()

In [ ]:
xy_scaled_emp, xy_scaled_pred = next(results_show_iter)

plot_results(xy_scaled_emp, xy_scaled_pred)

In [ ]:
scope = y_pred_rescaled.max(axis=1) - y_pred_rescaled.min(axis=1)
scope_perc = scope / y_pred_rescaled.mean(axis=1)

np.median(scope_perc)

In [ ]:
order = np.argsort(scope)[::-1]
iter_order = iter(order)

In [ ]:
max_scope = next(iter_order)
plot_results(*one_page_results(max_scope))